<a href="https://colab.research.google.com/github/leehon94/Automated-Product-Identification_PJS/blob/master/Keras%20Models/Visualization_of_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.applications.nasnet import preprocess_input
from keras.preprocessing.image import load_img
from keras.models import load_model

from keras import preprocessing
from keras import backend as K
from keras import models

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np


In [6]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Load Models

## NASNest


In [0]:
%cd '/content/gdrive/My Drive/'
image_path = 'MVP Datensatz/Testbilder/Art_1_01.JPG'
model_path = 'Keras Models/NASNet_5-1.h5'

img_width = 224
img_height = 224

In [0]:
# NASNet 

from keras.applications import NASNetMobile
base_model = NASNetMobile(
  include_top = False,
  weights = 'imagenet',
  input_shape = (img_height,img_width,3) 
 )

base_model.summary()

In [0]:
model = load_model(model_path)

model.summary()

##VGG19

In [10]:
%cd '/content/gdrive/My Drive/'
image_path = 'MVP Datensatz/Testbilder/Art_1_01.JPG'
model_path = 'Keras Models/VGG19_6-1.h5'

img_width = 224
img_height = 224

/content/gdrive/My Drive


In [0]:
# VGG19 

from keras.applications import VGG19
base_model = VGG19(
  include_top = False,
  weights = 'imagenet',
  input_shape = (img_height,img_width,3) 
 )

base_model.summary()

In [12]:
model = load_model(model_path)

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

## Model and Image

In [13]:
# Load pre-trained Keras model and the image to classify
image = load_img(image_path, target_size=(img_height, img_width))
img_tensor = preprocessing.image.img_to_array(image)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor = preprocess_input(img_tensor)

input_layer = model.get_layer("model_input")
conv_layer = model.get_layer("block5_conv4")
#heatmap_model = models.Model([input_layer.output, model.inputs], [conv_layer.output, model.output])
heatmal_model = models.Model()

ValueError: ignored

In [0]:
# Get gradient of the winner class w.r.t. the output of the (last) conv. layer
with tf.GradientTape() as gtape:
    conv_output, predictions = heatmap_model(img_tensor)
    loss = predictions[:, np.argmax(predictions[0])]
    grads = gtape.gradient(loss, conv_output)
    pooled_grads = K.mean(grads, axis=(0, 1, 2))

# Get values of pooled grads and model conv. layer output as Numpy arrays
iterate = K.function([model.inputs], [pooled_grads, conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value = iterate([img_tensor])

# Multiply each channel in the feature-map array by "how important it is"
for i in range(pooled_grads_value.shape[0]):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]



In [0]:
# Channel-wise mean of resulting feature-map is the heatmap of class activation
heatmap = np.mean(conv_layer_output_value, axis=-1)
heatmap = np.maximum(heatmap, 0)
max_heat = np.max(heatmap)
if max_heat == 0:
    max_heat = 1e-10
heatmap /= max_heat



In [0]:
# Render heatmap via pyplot
plt.matshow(heatmap)
plt.show()